<a href="https://colab.research.google.com/github/phiiaz/Tutorial/blob/master/Java_Collections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Java Kernel for Google Colab
This notebook is a template to use Java in Google Colaboratory (inspired by [Google Colab Javascript QuickStart](https://dev.to/obenjiro/silence-of-the-fans-part-1-javascript-quickstart-5f3m)). It uses [IJava](https://github.com/SpencerPark/IJava) as the Jupyter Kernel.
1. Run the cell bellow (click it and press `Shift`+`Enter`),
2. then refresh the page (press `F5`) right after that.

Now you can write Java code. See examples in the next section.

In [ ]:
!apt update -q
!apt-get install -q openjdk-11-jdk-headless
!curl -L https://github.com/SpencerPark/IJava/releases/download/v1.3.0/ijava-1.3.0.zip -o ijava-kernel.zip
!unzip -q ijava-kernel.zip -d ijava-kernel && cd ijava-kernel && python3 install.py --sys-prefix
!jupyter kernelspec list

In [ ]:
System.out.println("Hello world!");

Hello world!


# Java Collections
